<div><center>
<table><tr>
<td>
<img src="images/Embraer_logo.png" alt="Empresa Brasileira de Aeronáutica (Embraer)" style="width: 300px;"/>
    </td>
<td>
<img src="images/ITA_logo.png" alt="Instituto Tecnológico da Aeronáutica (ITA)" style="width: 300px;"/>
    </td>
    </tr></table
</center></div>

# EMBRAER | ITA

## ENGENHARIA AERONÁUTICA E MECÂNICA - PEE 30

### AC-701 : SISTEMAS PROPULSIVOS - PROF. LACAVA

### Autores:

* Camilla Santos
* Eloi Antonio
* Gabriel Henrique
* Jonas Degrave
* Lucas Schroeder
* Matheus Monteverde
* Orlando Gabriel
* Rafael Bayão

---

## Exercício 1: Montagem do Simulador

* Matheus Monteverde
* Rafael Bayão

### Estratégia para criação do simulador

O simulador foi montado a partir de uma arquitetura de Programação Orientada a Objetos. Dessa forma, foram modelados processos que são herdados pelos componentes e esses são utilizados pelos motores a medida da necessidade. Os a lista de objetos e suas funções podem ser visualizados abaixo:

Processos térmicos: cada processo armazena as propriedades adequadas para formulação de suas transformações.

* ThermalProcess: Classe principal que implementa métodos genéricos isentrópicos e é herdada pelas demais.
* StaticThermalProcess: implementa métodos para processos com entrada e saída com velocidades desprezíveis para o gás.
* SpeedInThermalProcess: implementa métodos para processos com entrada com velocidade considerável e saída com velocidade desprezível para o gás.
* SpeedOutThermalProcess: implementa métodos para processos com entrada com velocidade desprezível e saída com velocidade consideravel para o gás.

Componentes: cada componente herda um processo térmico adequado ao seu funcionamento implementa as equações necessárias. Os valores de entrada são armazenados e os de saída calculados ao serem invocados. Por fim, implementam métodos para variar a rotação e resumir todos os parametros inclusos.

* Diffuser_Adiab
* Nozzle_Adiab
* Compressor
* Fan
* Turbine
* FanTurbine
* FreeTurbine
* CombustionChamber

Motores: cada motor instancia sequencialmente os componentes necessários para o funcionamento e implementa métodos para ajustar globalmente a rotação e fornecer todas as propriedades de entrada e saída dos componentes e do próprio motor.

* TurboJet
* TurboFan
* TurboProp

### Exemplos resolvidos

#### Importação de pacotes

In [1]:
import model_reference as mf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

plt.style.use('seaborn')
warnings.filterwarnings('ignore')

#### Exemplo 2.1: TurboJet

Informações do exercício:

Calcular o empuxo específico e consumo específico (TSFC) para um motor turbojato operando a Mach 0,85 e a 12.200 m (Pa = 18,75kPa e Ta = 216,7 K). Usar os dados abaixo para eficiências e razões de calor específico.

| Componente | Eficiência Adiabática| $\gamma$            |
|------------|----------------------|-------------------  |
| difusor    | 0,97                 | 1,40                |
| compressor | 0,85                 | 1,37                |
| combustão  | 1,00                 | 1,35                |
| turbina    | 0.90                 | 1,33                |
| bocal      | 0,98                 | 1,36                |

Dados operacionais


| Dados operacionais:                           | Valor             |
|-----------------------------------------------|-------------------|
| Razão de Pressão no compressor                | 30                |
| Temperatura na sáida da câmara de combustão   | 1600 K            |
| Poder Calorífico Inferior do Combustível      | 45000 kJ/kg       |
| R Médio                                       | 288,3 m^2/(s^2.K) |
| Cp no combustor                               | 1,11 kJ/(kg.K)    |

##### Solução

In [7]:
values_ex2_1 = {
    'ta':216.7,         # Temperatura ambiente
    'pa':18.750,        # Pressão atmosférica
    'gamma_d':1.4,      # gamma no difusor
    'gamma_c':1.37,     # gamma no compressor
    'gamma_b':1.35,     # gamma na combustão
    'gamma_t':1.33,     # gamma na turbina
    'gamma_n':1.36,     # gamma no bocal
    'n_d':0.97,         # gamma no difusor
    'n_c':0.85,         # gamma no compressor
    'n_b':1,            # gamma na combustão
    'n_t':0.9,          # gamma na turbina
    'n_n':0.98,         # gamma no bocal
    'prc':30,           # Taxa de compressão - compressor
    'pc_fuel':45000,    # Poder Calorífico do combustível
    'cp_fuel':1.110,    # Capacidade calorífica do ar no combustor
    'r':288.3,          # R médio do ar
    't04':1600,         # Temperatura na saída da camera de combustão
    'mach':0.85         # mach de voo
    }

turbojet = mf.TurboJet(values_ex2_1)    # Instanciando o motor
targets = ['t02','p02','t03', 'p03', 'f', 't05', 'p05', 'u_s', 'TSFC', 'u_i', 'specific_thrust']    # Definindo variáveis de interesse
result_2_1 = (
    turbojet.sumarise().
    append(turbojet.sumarise_results()).
    loc[targets]
)
clean_names = ['Temperatura na Saída Difusor', 'Pressão na Saída Difusor', 
'Temperatura na Saída Compressor', 'Pressão na Saída Compressor', 
'Razão combustível/ar', 'Temperatura na Saída Turbina', 
'Pressão na Saída Turbina', 'Velocidade de Saída do ar', 
'Consumo específico', 'Velocidade de voo', 'Empuxo específico']
result_2_1.columns = ['Valores']
result_2_1.rename(index = dict(zip(targets, clean_names)))

,Valores
Temperatura na Saída Difusor,248.013150
Pressão na Saída Difusor,29.674830
Temperatura na Saída Compressor,687.349061
Pressão na Saída Compressor,890.244901
Razão combustível/ar,0.023437
Temperatura na Saída Turbina,1160.664089
Pressão na Saída Turbina,205.315935
Velocidade de Saída do ar,1078.304658
Consumo específico,0.027502
Velocidade de voo,251.382314


#### Comparação de resultados

| Parâmetro                                                 | Modelo            | Referência do material    | $\Delta$  |
|-----------------------------------------------------------|-------------------|---------------------------|-----------|
|Temp. total no final da entrada de ar                      | 248.013 K         | 248,013 K                 | 0 K       |
|Press. total no final da entrada de ar                     | 296.7483 kPa      | 29,6748 kPa               | 0 kPa     |
|Press. Total na saída do Compressor                        | 890,245 kPa       | 890,245 kPa               | 0 kPa     |
|Temp. total na saída do Compressor                         | 687,349 K         | 687,349 K                 | 0 K       |
|Razão Combustível/Ar                                       | 0,02344           | 0,02344                   | 0         |
|Temp. total na saída da turbina do compressor              |1160,66 K          | 1160,66 K                 | 0 K       |
|Press. total na saída da turbina do compressor             |205,316 kPa        |205,316 kPa                | 0 kPa     |
|Velocidade de saída                                        |1078,3 m/s         | 1078,3 m/s                | 0 m/s     |
|Consumo Específico                                         | 0,0275 kg/(kN.s)  | 0,0275 kg/(kN.s)          | 0 kg/(kN.s)|
|Velocidade de vôo                                          |251,382 m/s        | 251,382 m/s               | 0 m/s     |
|Empuxo Específico                                          |0,85219 kN.s/kg    | 0,85219 kN.s/kg           | 0 kN.s/kg |


## Exercício 2: Conceito da Operação do Motor Turbofan (ponto de projeto)

* Lucas Schroeder
* Jonas Degrave

## Exercício 3: Evolução dos Motores Turbofan

* Eloi Antonio
* Gabriel Henrique

## Exercício 4: Operação dos Motores Turbo Hélice

* Camilla Santos
* Orlando Gabriel

# RELATÓRIO TÉCNICO

In [1]:
# Importing needed libraries:
import numpy
import matplotlib as plt

In [1]:
print("Hello, World!")

Hello, World!


In [1]:
for i in range(10):
    print("i: {}".format(i))

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
